In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

from dosipy.utils.viz import fig_config, save_fig
from utils import *

In [ ]:
def take_a_look(xt, yt, zt):
    fig_config(latex=True, scaler=1, text_size=16)
    fig = plt.figure()
    ax = plt.axes(projection ='3d')
    ax.scatter(xt, yt, zt, s=10, color='k', depthshade=True)
    ax.plot(xs[:int(xs.size/2)-50], ys[:int(xs.size/2)-50], zs[:int(xs.size/2)-50], linestyle='-', c='k')
    ax.plot(xs[int(xs.size/2)+50:], ys[int(xs.size/2)+50:], zs[int(xs.size/2)+50:], linestyle='-', c='k')
    ax.set(xlabel='$x$ [mm]', ylabel='$y$ [mm]', zlabel='$z$ [mm]',
           xticks=[xt.min(), 0.0, xt.max()],
           yticks=[h, h/2, 0.0],
           zticks=[zt.min(), 0.0, zt.max()],
           xticklabels=[round(xt.min()*1000, 1), 0.0, round(xt.max()*1000, 1)],
           yticklabels=['$d$', '$d/2$', 0.0],
           zticklabels=[round(zt.min()*1000, 1), 0.0, round(zt.max()*1000, 1)],)
    ax.xaxis.pane.fill = False
    ax.yaxis.pane.fill = False
    ax.zaxis.pane.fill = False
    ax.xaxis.labelpad = 10
    ax.yaxis.labelpad = 10
    ax.zaxis.labelpad = 10
    ax.view_init(25, -40)
    fig.tight_layout()
    return fig, ax

In [ ]:
# frequency
f = 6

# separation distance
h = -2.5 / 1000

# averaging surface
if f < 30:
    target_area = (0.02, 0.02)
else:
    target_area = (0.01, 0.01)
A = target_area[0] * target_area[1]
target_area_origin = (-target_area[0]/2, -target_area[1]/2)

# source
xs = np.load(os.path.join('data', f'x_at{f}GHz.npy'))
xs = xs - xs.max() / 2
xs = np.asarray(xs.flatten())
ys = np.zeros_like(xs) + h
zs = np.zeros_like(xs)

In [ ]:
# planar target
N = 11
x = np.linspace(-target_area[0] / 2, target_area[0] / 2, N)
y = 0.0
z = np.linspace(-target_area[1] / 2, target_area[1] / 2, N)
Xt, Zt = np.meshgrid(x, z)
xt_pln = Xt.ravel()
yt_pln = np.zeros_like(xt_pln)
zt_pln = Zt.ravel()

fig, ax = take_a_look(xt_pln, yt_pln, zt_pln)

#fname = os.path.join('figures', 'planar_surface')
#save_fig(fig, fname, formats=['png'])

In [ ]:
# spherical target
N = 131 if f < 30 else 302
theta = np.linspace(0, np.pi, N)
phi = np.linspace(-np.pi, 0, N)
r = 0.0725
Theta, Phi = np.meshgrid(theta, phi)

x_sph, y_sph, z_sph = sph2cart(r, Theta.ravel(), Phi.ravel())
xyz_sph = jnp.c_[x_sph, y_sph, z_sph]
xyz_sph = xyz_sph[(xyz_sph[:, 0] >= xt_pln.min())
                  & (xyz_sph[:, 0] <= xt_pln.max())
                  & (xyz_sph[:, 2] >= zt_pln.min())
                  & (xyz_sph[:, 2] <= xt_pln.max())]
xt_sph = xyz_sph[:, 0]
yt_sph = xyz_sph[:, 1] - xyz_sph[:, 1].min()
zt_sph = xyz_sph[:, 2]

fig, ax = take_a_look(xt_sph, yt_sph, zt_sph)

#fname = os.path.join('figures', 'spherical_surface')
#save_fig(fig, fname, formats=['png'])

In [ ]:
# cylindrical target
N = 121 if f < 30 else 302
theta = np.linspace(0, np.pi, N)
Theta, Zt = np.meshgrid(-theta, z)

x_cyl, y_cyl, z_cyl = cyl2cart(r, Theta.ravel(), Zt.ravel())
xyz_cyl = jnp.c_[x_cyl, y_cyl, z_cyl]
xyz_cyl = xyz_cyl[(xyz_cyl[:, 0] >= xt_pln.min())
                  & (xyz_cyl[:, 0] <= xt_pln.max())]
xt_cyl = xyz_cyl[:, 0]
yt_cyl = xyz_cyl[:, 1] - xyz_cyl[:, 1].min()
zt_cyl = xyz_cyl[:, 2]

fig, ax = take_a_look(xt_cyl, yt_cyl, zt_cyl)

#fname = os.path.join('figures', 'cylindrical_surface')
#save_fig(fig, fname, formats=['png'])